In [301]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple, List
import os

In [229]:
@f.udf(returnType=t.FloatType())
def converter_minutos(tempo: str) -> float:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_segundos = duracao.total_seconds() 
    total_minutos = total_segundos / 60
   
    return round(total_minutos, 2)


In [230]:
converter_minutos('PT57S')

Column<'converter_minutos(PT57S)'>

In [231]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

In [232]:
def carregar_dataframe_prata(spark_session : SparkSession, assunto: str, metrica: str = None):
    if metrica is not None:
        df_cities_skylines = spark_session \
            .read \
                .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/{metrica}/*.parquet')
    else:
        df_cities_skylines = spark_session \
            .read \
                .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/*.parquet')
    return df_cities_skylines

In [233]:
def criar_particao_ouro(dataframe: DataFrame, assunto_pesquisa: str, metrica: str,  nome_arquivo: str, colunas_particao: Tuple = None ):
    caminho_datalake_ouro = '/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/'
    if colunas_particao == None:
        dataframe.write. \
            parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
    else: 
        if isinstance(colunas_particao, Tuple) :
            dataframe.write. \
                partitionBy(*colunas_particao). \
                parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
        else:
            dataframe.write. \
                partitionBy(colunas_particao). \
                parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))


In [234]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        f.explode('DATA_PUBLICACAO').alias('DATA_PUBLICACAO'),
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                f.explode('ID_CANAL').alias('ID_CANAL'),
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                f.explode('NM_CANAL').alias('NM_CANAL'), 
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                f.explode('ID_CATEGORIA').alias('ID_CATEGORIA'),
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                f.explode('ID_VIDEO').alias('ID_VIDEO'),
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                f.explode('TITULO_VIDEO').alias('TITULO_VIDEO'),
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'ID_VIDEO',
                f.explode('DESCRICAO').alias('DESCRICAO'),
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                f.explode('TAGS').alias('TAGS'),
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                f.explode('DURACAO_VIDEOS').alias('DURACAO_VIDEOS'),
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                f.explode('TOTAL_VISUALIZACOES').alias('TOTAL_VISUALIZACOES'),
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                f.explode('TOTAL_LIKES').alias('TOTAL_LIKES'),
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                f.explode('TOTAL_FAVORITOS').alias('TOTAL_FAVORITOS'),
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                f.explode('TOTAL_COMENTARIOS').alias('TOTAL_COMENTARIOS'),
        )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_SEGUNDOS', converter_minutos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    return dataframe_transform
  

In [235]:
@f.udf(returnType=t.StringType())
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [236]:
@f.udf(returnType=t.IntegerType())
def indice_semana(dia):
    traducao = {
        'Domingo': 1,
        'Segunda-feira': 2,
        'Terça-feira': 3,
        'Quarta-feira': 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado': 7
    }
    return traducao.get(dia, dia)

In [237]:
df_estatisticas_teste = carregar_dataframe_prata(spark_session=spark, assunto='assunto_cities_skylines', metrica='estatisticas')
df_estatisticas_teste.show()

+-------------------+-------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+
|      data_extracao|     ID_VIDEO|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|
+-------------------+-------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+
|               null|         null|                null|                null|                null|        null|                null|                null|                null|          null|               null|       null|           

In [238]:
df_estatisticas_teste = carregar_dataframe_prata(spark_session=spark, assunto='assunto_cities_skylines', metrica='estatisticas')
df_estatisticas_teste = transformacao_dataframe(df_estatisticas_teste)
df_estatisticas_teste.show(truncate=False)

+-------------+--------------------+------------------------+----------------------------+------------+-----------+------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<p>Depara Canal Vídeo</p>

In [239]:
def construir_depara_canal(dataframe: DataFrame, assunto: str):
    dataframe.select(
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO
    ).distinct().sort('ID_CANAL').write.\
        parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_{assunto}/depara_canal.parquet')


In [243]:
def obter_total_publicacao_video(dataframe : DataFrame) -> DataFrame:
   dataframe =  dataframe.select(
    'DATA_PUBLICACAO',
    'ID_VIDEO'
    ).withColumn(
       'DATA_PUBLICACAO', f.date_format('DATA_PUBLICACAO', 'yyyy-MM-dd')
    ) \
        .sort('ID_VIDEO') \
        .withColumn(
            'SEMANA_TRADUZIDA', 
            traduzir_dia_da_semana(
                f.date_format('DATA_PUBLICACAO', 'EEEE')
            )
            ).distinct() \
                .groupBy('DATA_PUBLICACAO','SEMANA_TRADUZIDA'). \
                    count(). \
                        alias('TOTAL_VIDEOS'). \
                            withColumn('INDICE_SEMANA', indice_semana('SEMANA_TRADUZIDA')) \
                    .sort('INDICE_SEMANA')\
                        .select('DATA_PUBLICACAO','SEMANA_TRADUZIDA', f.col('TOTAL_VIDEOS.COUNT').alias('TOTAL_VIDEOS'))       
   return dataframe

In [244]:
obter_total_publicacao_video(df_estatisticas_teste).show()

+---------------+----------------+------------+
|DATA_PUBLICACAO|SEMANA_TRADUZIDA|TOTAL_VIDEOS|
+---------------+----------------+------------+
|     2023-10-15|         Domingo|          12|
|     2023-10-22|         Domingo|          21|
|     2023-10-16|   Segunda-feira|          16|
|     2023-10-23|   Segunda-feira|          25|
|     2023-10-17|     Terça-feira|          18|
|     2023-10-24|     Terça-feira|          27|
|     2023-10-18|    Quarta-feira|          15|
|     2023-10-25|    Quarta-feira|          25|
|     2023-10-26|    Quinta-feira|          27|
|     2023-10-19|    Quinta-feira|          20|
|     2023-10-27|     Sexta-feira|          23|
|     2023-10-20|     Sexta-feira|          14|
|     2023-10-21|          Sábado|          25|
+---------------+----------------+------------+



<p>ETL Para obter o total de vídeos públicado por semaba</p>

In [245]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_publicacao_video(df_estatisticas_etl)
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_video_publicado_semana',
        nome_arquivo='total_video_publicado_semana.parquet',
    )
    
    

ETL PARA  Power BI


ETL PARA  Python AND dados
ETL PARA  Cities Skylines
ETL PARA  Cities Skylines 2


<p>Dia da semana com mais visualização</p>

In [246]:
@f.udf(returnType=t.StringType())
def semana(indice: int) -> str:
    semana = {
        1: 'Domingo',
        2: 'Segunda-feira',
        3: 'Terça-feira',
        4: 'Quarta-feira',
        5: 'Quinta-feira',
        6: 'Sexta-feira',
        7: 'Sábado'
    }
    return semana.get(indice, indice)
    

<p>Vísualizações por semana</p>

In [302]:
def obter_total_visualizacoes_semana(
        dataframe: DataFrame, 
        colunas_agrupamento: Tuple, 
        lista_campos_agupamento: List[str]
        ) -> DataFrame:

    op_agrupamento = [ 
        f.max(metrica).alias(metrica) 
        for metrica in lista_campos_agupamento
        ]
    df_estatisticas_etl = dataframe \
        .groupBy(*colunas_agrupamento) \
        .agg(
            *op_agrupamento
        ) \
            
    return df_estatisticas_etl

In [303]:
obter_total_visualizacoes_semana(
    dataframe=df_estatisticas_teste,
    colunas_agrupamento=('data_hora_extracao', 'data_extracao', 'NM_CANAL', 'TITULO_VIDEO', 'ID_VIDEO'),
    lista_campos_agupamento=['TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS'],

).show()

+-------------------+-------------+--------------------+--------------------+-----------+-------------------+-----------------+
| data_hora_extracao|data_extracao|            NM_CANAL|        TITULO_VIDEO|   ID_VIDEO|TOTAL_VISUALIZACOES|TOTAL_COMENTARIOS|
+-------------------+-------------+--------------------+--------------------+-----------+-------------------+-----------------+
|2023-10-27 08:15:28|   2023-10-27|            Jowgando|Live - Cities Sky...|DVJOF93OsTw|               1134|                7|
|2023-10-27 08:15:37|   2023-10-27|CANAL HAND - GAME...|CITIES SKYLINES |...|f2oVDmsT2fU|                 10|                0|
|2023-10-27 09:00:43|   2023-10-27|            MalvinaN|🎭[MTA] My Last H...|r_2IEGjYcv4|                336|                3|
|2023-10-27 11:00:41|   2023-10-27|       Irmãos Cities|Cities Skylines, ...|YhXTcceM1NA|                586|               25|
|2023-10-27 11:00:42|   2023-10-27|           MetalBear|ESPERE pra compra...|vzzeZU07rmo|              16

In [288]:
df_estatisticas_teste.show(3)

+-------------+--------------------+--------------------+---------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|       NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_SEGUNDOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|
+-------------+--------------------+--------------------+---------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|   2023-10-27|2023-10-24T00:50:04Z|UCNv3_K

In [294]:
a = [1, 2]
print(*a)

1 2


In [300]:
lista = ['a', 'b']

teste = [f.max(i).alias(i) for i in lista]
teste

[Column<'max(a) AS a'>, Column<'max(b) AS b'>]

In [299]:

df_estatisticas_teste\
    .groupBy('data_extracao','ID_VIDEO', 'TITULO_VIDEO') \
    .agg(
        *teste
        ) \
    .filter(df_estatisticas_teste.ID_VIDEO == 'rZX5J7Xugus') \
    .sort('data_extracao', 'TITULO_VIDEO').show(truncate=False)

+-------------+-----------+---------------------------------------+-------------------+-----------------+
|data_extracao|ID_VIDEO   |TITULO_VIDEO                           |TOTAL_VISUALIZACOES|TOTAL_COMENTARIOS|
+-------------+-----------+---------------------------------------+-------------------+-----------------+
|2023-10-15   |rZX5J7Xugus|Cidade sem mods - Cities:Skylines [PS5]|398                |3                |
|2023-10-16   |rZX5J7Xugus|Cidade sem mods - Cities:Skylines [PS5]|575                |5                |
|2023-10-19   |rZX5J7Xugus|Cidade sem mods - Cities:Skylines [PS5]|633                |5                |
|2023-10-20   |rZX5J7Xugus|Cidade sem mods - Cities:Skylines [PS5]|634                |5                |
|2023-10-21   |rZX5J7Xugus|Cidade sem mods - Cities:Skylines [PS5]|640                |5                |
|2023-10-22   |rZX5J7Xugus|Cidade sem mods - Cities:Skylines [PS5]|648                |5                |
|2023-10-23   |rZX5J7Xugus|Cidade sem mods - C

<p>ETL Para visializações por semana<P>

In [251]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_visualizacoes_semana(
        dataframe=df_estatisticas_teste,
        colunas_agrupamento=('data_extracao', 'ID_CANAL', 'NM_CANAL', 'TITULO_VIDEO', 'ID_VIDEO'),
        lista_campos_agupamento=['TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_FAVORITOS', 'TOTAL_COMENTARIOS'],
   
    )
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_visualizacao_por_semana',
        colunas_particao=('data_extracao','ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='total_visualizacao_por_semana.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


In [ ]:
obter_total_visualizacoes_semana(
    dataframe=df_estatisticas_teste,
    colunas_agrupamento=('data_hora_extracao', 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO', 'TOTAL_VISUALIZACOES'),
    metrica_agrupamento='TOTAL_VISUALIZACOES'
).show()

+-------------------+--------------------+--------------------+-----------+--------------------+-------------------+-----------------------+
| data_hora_extracao|            ID_CANAL|            NM_CANAL|   ID_VIDEO|        TITULO_VIDEO|TOTAL_VISUALIZACOES|max(data_hora_extracao)|
+-------------------+--------------------+--------------------+-----------+--------------------+-------------------+-----------------------+
|2023-10-15 14:50:31|UCEc5jA0NvvR-yJC0...|              GAC TV|igxEHvMej3g|EQUILIBRANDO A CI...|                  2|    2023-10-15 14:50:31|
|2023-10-15 14:50:31|UCTrNnXivZU_1-aJq...|              DILSÃO|Gu_MQ8RJ7f4|Começando do Zero...|                 13|    2023-10-15 14:50:31|
|2023-10-15 14:50:31|UCmg3_taw1BrGuo1I...|        O Construtor|X4v776WzZQY|Metrópole sem mod...|                279|    2023-10-15 14:50:31|
|2023-10-15 14:50:31|UCrOH1V-FyMunBIMr...|     ChratosGameplay|XOUzWJ0bHuw|MODDERS decidem f...|               1676|    2023-10-15 14:50:31|
|2023-10-15 1

<p>Comentários</p>

In [252]:
df_comentarios = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_power_bi', 
        metrica='comentarios'
)

In [253]:
def extrair_comentarios(dataframe: DataFrame) -> DataFrame:
    dataframe = dataframe \
        .select('ID_CANAL', 'ID_COMENTARIO', 'ID_VIDEO', 'TEXTO_COMENTARIO')  \
        .distinct().sort('ID_VIDEO') 
    return dataframe
# Ugw-AOQ6Kh-Qof2U9bd4AaABAg - Gl60rzPP9VQ

In [254]:
extrair_comentarios(df_comentarios).show(truncate=False)

+------------------------+--------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID_CANAL                |ID_COMENTARIO             |ID_VIDEO   |TEXTO_COMENTARIO                                                                                                                                                                                                   |
+------------------------+--------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|UCg6yqeJJ2WMFVt9bXeUd-zA|UgwcmMNJM8lHN6YKgF94AaABAg|-Jg3j6EH7cg|Um tecnólogo com conhecimentos técnicos e proficiência em idiomas é suficiente, desde que também poss

<p>Extacao comentário</p>

In [255]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_comentarios = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='comentarios'
    )
    df_comentarios = extrair_comentarios(df_comentarios)
    criar_particao_ouro(
        dataframe=df_comentarios, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='comentarios',
        colunas_particao=('ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='comentarios.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


In [256]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()

    df_resposta_comentarios = carregar_dataframe_prata(
        assunto=f'assunto_{assunto}',
        spark_session=spark,
        metrica='resposta_comentarios'
    )   


    df_resposta_comentarios = df_resposta_comentarios.select(
        'ID_CANAL', 
        'ID_RESPOSTA_COMENTARIOS', 
        'TEXTO').withColumnRenamed(
            'ID_CANAL', 'ID_CANAL'
        ).distinct()
    criar_particao_ouro(
        dataframe=df_resposta_comentarios, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='reposta_comentarios',
        colunas_particao=('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS'),
        nome_arquivo='reposta_comentarios.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


In [257]:
df_resposta_comentarios.select('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS', 'TEXTO').distinct().show(truncate=False)

+------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID_CANAL                |ID_RESPOSTA_COMENTARIOS                          |TEXTO                                                                                                                                                                                                                                                                                                                         |
+------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------

<p>Trends do Youtube</p>

In [258]:
df_trends = carregar_dataframe_prata(
    spark_session=spark,
    assunto='top_brazil',
)

df_trends.show()

+-------------------+------------+--------------------+------------------+-----------+----------------------+--------+-------------------------+-----------------+---------------+-----------+-------------------+
|      data_extracao|ID_CATEGORIA|            ID_CANAL|          NM_CANAL|   ID_VIDEO|          TITULO_VIDEO| DURACAO|                DESCRICAO|TOTAL_COMENTARIOS|TOTAL_FAVORITOS|TOTAL_LIKES|TOTAL_VISUALIZACOES|
+-------------------+------------+--------------------+------------------+-----------+----------------------+--------+-------------------------+-----------------+---------------+-----------+-------------------+
|2023-10-23 08:32:43|          20|UCHuZmQ0lSW8TG81d...|        LipaoGamer|qvgmMih8FOc|  SPIDER MAN 2 PS5 ...|PT26M59S|     Vídeos todos os d...|              295|              0|      17428|             500862|
|2023-10-23 08:32:43|          17|UCv-Nx8pSfG_LxbVi...|Jovem Pan Esportes|Rq1RqHl2AO0|  "CADÊ O 'PLANO' N...|PT21M28S|     CRISE NO PALMEIRA...|            

In [259]:
df_trends = df_trends.withColumn('data_hora_extracao', df_trends['data_extracao'])
df_trends = df_trends.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
df_trends = df_trends.withColumn('DURACAO', converter_minutos(f.col('DURACAO')))
df_trends = df_trends.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))


In [260]:
df_trends.show()

+-------------+------------+--------------------+------------------+-----------+----------------------+-------+-------------------------+-----------------+---------------+-----------+-------------------+-------------------+
|data_extracao|ID_CATEGORIA|            ID_CANAL|          NM_CANAL|   ID_VIDEO|          TITULO_VIDEO|DURACAO|                DESCRICAO|TOTAL_COMENTARIOS|TOTAL_FAVORITOS|TOTAL_LIKES|TOTAL_VISUALIZACOES| data_hora_extracao|
+-------------+------------+--------------------+------------------+-----------+----------------------+-------+-------------------------+-----------------+---------------+-----------+-------------------+-------------------+
|   2023-10-23|          20|UCHuZmQ0lSW8TG81d...|        LipaoGamer|qvgmMih8FOc|  SPIDER MAN 2 PS5 ...|  26.98|     Vídeos todos os d...|              295|              0|      17428|             500862|2023-10-23 08:32:43|
|   2023-10-23|          17|UCv-Nx8pSfG_LxbVi...|Jovem Pan Esportes|Rq1RqHl2AO0|  "CADÊ O 'PLANO' N...| 

In [261]:
df_trends.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- ID_CATEGORIA: string (nullable = true)
 |-- ID_CANAL: string (nullable = true)
 |-- NM_CANAL: string (nullable = true)
 |-- ID_VIDEO: string (nullable = true)
 |-- TITULO_VIDEO: string (nullable = true)
 |-- DURACAO: float (nullable = true)
 |-- DESCRICAO: string (nullable = true)
 |-- TOTAL_COMENTARIOS: string (nullable = true)
 |-- TOTAL_FAVORITOS: string (nullable = true)
 |-- TOTAL_LIKES: string (nullable = true)
 |-- TOTAL_VISUALIZACOES: integer (nullable = true)
 |-- data_hora_extracao: string (nullable = true)



In [286]:
df_total_vis_sem_trend = obter_total_visualizacoes_semana(
    dataframe=df_trends,
    colunas_agrupamento=('data_extracao', 'ID_CATEGORIA','ID_CANAL', 'NM_CANAL',  'ID_VIDEO', 'TITULO_VIDEO', 'TOTAL_LIKES'),
    lista_campos_agupamento=['TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_FAVORITOS', 'TOTAL_COMENTARIOS'],
    )

df_total_vis_sem_trend.show()

+-------------+------------+--------------------+--------------------+-----------+--------------------+-----------+-------------------+
|data_extracao|ID_CATEGORIA|            ID_CANAL|            NM_CANAL|   ID_VIDEO|        TITULO_VIDEO|TOTAL_LIKES|TOTAL_VISUALIZACOES|
+-------------+------------+--------------------+--------------------+-----------+--------------------+-----------+-------------------+
|   2023-10-23|          20|UCooVYzDxdwTtGYAk...|         Brawl Stars|QMBF9dvMyO8|Brawl Stars: Braw...|     255365|            4072165|
|   2023-10-23|          20|UCn-0dkyNA4pPMrBf...|      Patricio Games|AEJM1nNKAcM|CORRE!! NOVO CÓDI...|       9462|             213169|
|   2023-10-23|          10|UCKd9e3BrkQ-dZdal...|       Marcos Baroni|kvngE7TfcOM|Melhor Dia 12 | L...|      19604|             116005|
|   2023-10-23|          26|UCuD2V9FCQiDTjTLZ...|     Maurício Arruda|ua1DsYjuavo|   TOUR NO MEU SÍTIO|      16792|             186481|
|   2023-10-23|          17|UCgCKagVhzGnZcuP9...

In [ ]:
criar_particao_ouro(
    dataframe=df_total_vis_sem_trend,
    assunto_pesquisa='trends_brazil',
    colunas_particao=('data_extracao', 'ID_CATEGORIA', 'ID_CANAL', 'ID_VIDEO'),
    metrica='trends_brazil',
    nome_arquivo='trends_brazil.parquet'
)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 3) / 3]
Traceback (most recent call last):
  File "/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

23/11/02 14:54:51 ERROR Utils: Aborting task
ExitCodeException exitCode=1: chmod: não foi possível acessar '/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/trends_brazil/trends_brazil/trends_brazil.parquet/_temporary/0/_temporary/attempt_202311021453388609699351119426120_0783_m_000000_1248/data_extracao=2023-10-21/ID_CATEGORIA=26/ID_CANAL=UCXBY-tNwbGkF5P__nN5IRbg/ID_VIDEO=HB8UKjr0N_Q/part-00000-ced314b0-ba34-48e8-8b21-bebb1c90b0ca.c000.snappy.parquet': Arquivo ou diretório inexistente

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem